<a href="https://colab.research.google.com/github/yuma-gri/QM2/blob/main/choropleths_updated_code_qm2_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
url = "https://data.london.gov.uk/download/v8pow/87e880c2-34bd-4d86-8895-e8c5344f358e/traffic-flow-borough.xlsx"

cars = pd.read_excel(url, sheet_name="Traffic Flows - Cars")
allv = pd.read_excel(url, sheet_name="Traffic Flows - All vehicles")

def clean_year(col):
    try:
        return int(col.split()[0])
    except:
        return col

cars.columns = [clean_year(c) for c in cars.columns]
allv.columns = [clean_year(c) for c in allv.columns]

years = list(range(2019, 2023))

cars_df = cars[["LA Code", "Local Authority"] + years]
vehicles_df = allv[["LA Code", "Local Authority"] + years]

In [ ]:
#Traffic Flow Car Output
cars_df

In [ ]:
#Traffic Flow All Vehicle Output
vehicles_df

In [ ]:
import pandas as pd
import requests
from io import BytesIO

# data in population
df_path_pop = (
    "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/"
    "populationandmigration/populationestimates/datasets/"
    "populationestimatesforukenglandandwalesscotlandandnorthernireland/"
    "mid2024/mye24tablesuk.xlsx"
)

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(df_path_pop, headers=headers)
response.raise_for_status()

pop_df = pd.read_excel(BytesIO(response.content), sheet_name="MYE5", header=7)


In [ ]:
#Output pandas in population
pop_filtered_df = pop_df.iloc[201:234]
def clean_pop_df_column_name(col_name):
    if isinstance(col_name, str) and col_name.startswith('Mid-'):
        try:
            return int(col_name.replace('Mid-', ''))
        except ValueError:
            return col_name # Return original if conversion fails
    return col_name

pop_filtered_df.columns = [clean_pop_df_column_name(col) for col in pop_filtered_df.columns]
pop_filtered_df

In [ ]:
#Output pandas in population
pop_filtered_df = pop_df.iloc[201:234]
def clean_pop_df_column_name(col_name):
    if isinstance(col_name, str) and col_name.startswith('Mid-'):
        try:
            return int(col_name.replace('Mid-', ''))
        except ValueError:
            return col_name # Return original if conversion fails
    return col_name

pop_filtered_df.columns = [clean_pop_df_column_name(col) for col in pop_filtered_df.columns]

desired_columns = [
    'Code',
    'Name',
    'Geography',
    'Area (sq km)',
    'Estimated Population mid-2022',
    '2022 people per sq. km',
    'Estimated Population mid-2019',
    '2019 people per sq. km'
]

pop_selected_columns_df = pop_filtered_df[desired_columns]
display(pop_selected_columns_df)

In [ ]:
# data in earnings
df_path_earnings = "https://data.london.gov.uk/download/2z0rk/1686ef1c-b169-442d-8877-e7e49788f668/earnings-residence-borough.xlsx"

earnings_df = pd.read_excel(df_path_earnings, sheet_name="Total, weekly")


In [ ]:
import pandas as pd

raw_df = pd.read_excel(df_path_earnings, sheet_name="Total, weekly", header=None)

# Extract the first two rows to be used as header information
header_row0 = raw_df.iloc[0] # Contains years (e.g., 2002, NaN, 2003, NaN)
header_row1 = raw_df.iloc[1] # Contains sub-headers (e.g., Code, Area, Pay (£), conf %)

# Construct new column names by combining the year and sub-header
new_columns = []
current_year = None

for i in range(len(header_row0)):
    year_val = header_row0.iloc[i]
    sub_header_val = header_row1.iloc[i]

    if i < 2: # Handle the first two columns ('Code', 'Area') specifically
        new_columns.append(str(year_val).strip())
    elif pd.isna(year_val): # If year is NaN, it's a sub-header like 'conf %' under a year
        if current_year is not None:
            new_columns.append(f"{current_year} {str(sub_header_val).strip()}")
        else:
            # This case implies a NaN year_val without a preceding year, which shouldn't happen for data columns
            new_columns.append(str(sub_header_val).strip()) # Fallback for safety
    else: # Year value is present (e.g., 2002, 2003, ...)
        current_year = int(year_val)
        new_columns.append(f"{current_year} {str(sub_header_val).strip()}")

# Create the earnings_clean DataFrame by taking data from the third row onwards
# and assigning the newly constructed column names.
earnings_clean = raw_df.iloc[2:].copy()
earnings_clean.columns = new_columns
earnings_clean = earnings_clean.reset_index(drop=True)

# Remove all 'conf %' columns
columns_to_drop = [col for col in earnings_clean.columns if 'conf %' in col]
earnings_clean = earnings_clean.drop(columns=columns_to_drop)

# Identify the columns for 'Pay (£)' for years 2011 to 2024
years_to_keep = list(range(2019, 2023))
pay_columns = [f"{year} Pay (£)" for year in years_to_keep]

# Ensure 'Code' and 'Area' are always kept
final_columns = ['Code', 'Area'] + pay_columns

# Filter earnings_clean to retain only these selected columns
earnings_clean = earnings_clean[final_columns]

earnings_clean_df = earnings_clean.iloc[0:34]

print("Cleaned earnings_clean DataFrame:")
earnings_clean_df

In [ ]:
import io
import zipfile
import requests
import pandas as pd

# Make pandas show EVERYTHING
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)


def load_full_pm25_summary(zip_url):
    """
    Downloads LAEI ZIP file and returns the FULL PM2.5 Summary sheet as a DataFrame
    """
    # Download ZIP
    response = requests.get(zip_url)
    response.raise_for_status()

    # Open ZIP
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # Find Excel file
        excel_name = [f for f in z.namelist() if f.lower().endswith(".xlsx")][0]

        # Load Excel
        excel_bytes = io.BytesIO(z.read(excel_name))
        df = pd.read_excel(excel_bytes, sheet_name="PM2.5 Summary")

    return df


# ================= URLs =================

url_2019 = (
    "https://data.london.gov.uk/download/"
    "london-atmospheric-emissions-inventory--laei--2019/"
    "17d21cd1-892e-4388-9fea-b48c1b61ee3c/"
    "LAEI-2019-Emissions-Summary-including-Forecast.zip"
)

url_2022 = (
    "https://data.london.gov.uk/download/2lg5g/4ql/"
    "LAEI2022-Emissions-Summary-Excel.zip"
)

# ================= Load FULL PM2.5 sheets =================

pm25_2019_full = load_full_pm25_summary(url_2019)
pm25_2022_full = load_full_pm25_summary(url_2022)


In [ ]:

result_2022 = pm25_2022_full.iloc[[6, 77], 11:]

result_2022

In [ ]:

result_2019 = pm25_2019_full.iloc[[6, 71], 11:]

result_2019

In [ ]:
!pip install geopandas

import requests
import zipfile
import io
import geopandas as gpd
import tempfile
import os

zip_url = "https://data.london.gov.uk/download/20od9/9ba8c833-6370-4b11-abdc-314aa020d5e0/statistical-gis-boundaries-london.zip"

# Download the zip file
response = requests.get(zip_url)
response.raise_for_status() # Raise an exception for bad status codes

# Create a temporary directory to extract the contents
with tempfile.TemporaryDirectory() as tmpdir:
    # Open the zip file from memory
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(tmpdir)

    # Find the shapefile (.shp) within the extracted files
    shp_file = None
    for root, dirs, files in os.walk(tmpdir):
        for file in files:
            if file.endswith(".shp"):
                shp_file = os.path.join(root, file)
                break
        if shp_file:
            break

    if shp_file:
        london_boroughs_gdf = gpd.read_file(shp_file)
        print("GeoDataFrame loaded successfully:")
        print(london_boroughs_gdf.head())
    else:
        print("No shapefile (.shp) found in the zip archive.")

In [ ]:
import plotly.express as px

#choropleth traffic

# Ensure borough names in vehicles_df match names in boroughs_gdf for merging
# Assuming 'Local Authority' in vehicles_df corresponds to 'name' in boroughs_gdf
vehicles_df_cleaned = vehicles_df.copy()
vehicles_df_cleaned = vehicles_df_cleaned.rename(columns={'Local Authority': 'name'})

# Filter out NaN rows and non-borough rows if necessary
vehicles_df_cleaned = vehicles_df_cleaned.dropna(subset=['name'])
vehicles_df_cleaned = vehicles_df_cleaned[~vehicles_df_cleaned['name'].isin(['London', 'England', 'Great Britain', 'North East', 'North West', 'Yorkshire and the Humber', 'East Midlands', 'West Midlands', 'East of England', 'South East', 'South West', 'Scotland', 'Wales'])]

# Merge traffic data with geospatial data
merged_traffic_gdf = london_boroughs_gdf.merge(vehicles_df_cleaned, left_on='LAD11NM', right_on='name', how='left')

# Drop rows where there's no traffic data after merge
merged_traffic_gdf = merged_traffic_gdf.dropna(subset=[2019])

px.choropleth( # plot a choropleth map using the plotly express (px) library
                merged_traffic_gdf, # load the dataframe
                geojson=merged_traffic_gdf.geometry, # Use the geometry column from the merged GeoDataFrame
                locations=merged_traffic_gdf.index, # Use the index as locations since geojson is provided
                color=2019, # Use the integer column name for 2019
                color_continuous_scale=px.colors.sequential.Viridis, # set the color scale to Viridis, a commonly used color scale
                range_color=[merged_traffic_gdf[2019].min(), merged_traffic_gdf[2019].max()], # Set range dynamically
                hover_name="name", # Show borough name on hover
                title="All Vehicle Traffic Flow in London Boroughs (2019)",
                height=700)

In [ ]:
#choropleth pollution 2019

import plotly.express as px

# Extract the borough names and pollution values from result_2019
# result_2019 is a 2x36 DataFrame, where the first row contains labels (borough names)
# and the second row contains the actual pollution values. The first column is 'Unnamed: 11' which holds 'Row Labels'/'Grand Total'.
# We want columns from index 1 onwards for both rows.

pollution_names = result_2019.iloc[0, 1:].tolist()
pollution_values = result_2019.iloc[1, 1:].tolist()

# Create a temporary DataFrame with cleaned names and values
df_pollution_2019 = pd.DataFrame({'name': pollution_names, '2019_pollution': pollution_values})

# Clean up names for merging with london_boroughs_gdf
name_mapping = {
    'City': 'City of London',
    'City of Westminster': 'Westminster',
    'Non GLA': None, # Mark for removal
    'Grand Total': None # Mark for removal
}

df_pollution_2019['name'] = df_pollution_2019['name'].replace(name_mapping)
df_pollution_2019 = df_pollution_2019.dropna(subset=['name']) # Remove rows where name became None

# Ensure the pollution data is numeric
df_pollution_2019['2019_pollution'] = pd.to_numeric(df_pollution_2019['2019_pollution'], errors='coerce')

# Merge pollution data with geospatial data
merged_pollution_2019_gdf = london_boroughs_gdf.merge(
    df_pollution_2019,
    left_on='LAD11NM',
    right_on='name',
    how='left'
)

# Drop rows where there's no pollution data after merge or conversion error
merged_pollution_2019_gdf = merged_pollution_2019_gdf.dropna(subset=['2019_pollution'])

px.choropleth( # plot a choropleth map using the plotly express (px) library
                merged_pollution_2019_gdf, # load the dataframe
                geojson=merged_pollution_2019_gdf.geometry, # Use the geometry column from the merged GeoDataFrame
                locations=merged_pollution_2019_gdf.index, # Use the index as locations since geojson is provided
                color='2019_pollution', # Use the new column name for 2019 pollution
                color_continuous_scale=px.colors.sequential.Viridis, # set the color scale to Viridis, a commonly used color scale
                range_color=[merged_pollution_2019_gdf['2019_pollution'].min(), merged_pollution_2019_gdf['2019_pollution'].max()], # Set range dynamically
                hover_name="name", # Show borough name on hover (from the merged dataframe)
                title="Pollution in London Boroughs (2019)",
                height=700)

In [ ]:
earnings_2019_df = earnings_clean_df[['Code', 'Area', '2019 Pay (£)']].copy()
earnings_2019_df.rename(columns={'Code': 'LA Code', 'Area': 'Local Authority'}, inplace=True)
earnings_2019_df.dropna(subset=['LA Code'], inplace=True)

print("Prepared 2019 earnings data:")
earnings_2019_df.head()


In [ ]:
import pandas as pd

raw_df = pd.read_excel(df_path_earnings, sheet_name="Total, weekly", header=None)

# Extract the first two rows to be used as header information
header_row0 = raw_df.iloc[0] # Contains years (e.g., 2002, NaN, 2003, NaN)
header_row1 = raw_df.iloc[1] # Contains sub-headers (e.g., Code, Area, Pay (£), conf %)

# Construct new column names by combining the year and sub-header
new_columns = []
current_year = None

for i in range(len(header_row0)):
    year_val = header_row0.iloc[i]
    sub_header_val = header_row1.iloc[i]

    if i < 2: # Handle the first two columns ('Code', 'Area') specifically
        new_columns.append(str(year_val).strip())
    elif pd.isna(year_val): # If year is NaN, it's a sub-header like 'conf %' under a year
        if current_year is not None:
            new_columns.append(f"{current_year} {str(sub_header_val).strip()}")
        else:
            # This case implies a NaN year_val without a preceding year, which shouldn't happen for data columns
            new_columns.append(str(sub_header_val).strip()) # Fallback for safety
    else: # Year value is present (e.g., 2002, 2003, ...)
        current_year = int(year_val)
        new_columns.append(f"{current_year} {str(sub_header_val).strip()}")

# Create the earnings_clean DataFrame by taking data from the third row onwards
# and assigning the newly constructed column names.
earnings_clean = raw_df.iloc[2:].copy()
earnings_clean.columns = new_columns
earnings_clean = earnings_clean.reset_index(drop=True)

# Remove all 'conf %' columns
columns_to_drop = [col for col in earnings_clean.columns if 'conf %' in col]
earnings_clean = earnings_clean.drop(columns=columns_to_drop)

# Identify the columns for 'Pay (£)' for years 2011 to 2024
years_to_keep = list(range(2019, 2023))
pay_columns = [f"{year} Pay (£)" for year in years_to_keep]

# Ensure 'Code' and 'Area' are always kept
final_columns = ['Code', 'Area'] + pay_columns

# Filter earnings_clean to retain only these selected columns
earnings_clean = earnings_clean[final_columns]

earnings_clean_df = earnings_clean.iloc[0:34]

# Original failing code starts here
earnings_2019_df = earnings_clean_df[['Code', 'Area', '2019 Pay (£)']].copy()
earnings_2019_df.rename(columns={'Code': 'LA Code', 'Area': 'Local Authority'}, inplace=True)
earnings_2019_df.dropna(subset=['LA Code'], inplace=True)

print("Prepared 2019 earnings data:")
earnings_2019_df.head()

In [ ]:
import plotly.express as px

#choropleth income


# Ensure borough names in vehicles_df match names in boroughs_gdf for merging
# Assuming 'Local Authority' in vehicles_df corresponds to 'name' in boroughs_gdf
earnings_2019_df_cleaned = earnings_2019_df.copy()
earnings_2019_df_cleaned = earnings_2019_df_cleaned.rename(columns={'Local Authority': 'name'})

# Filter out NaN rows and non-borough rows if necessary
earnings_2019_df_cleaned = earnings_2019_df_cleaned.dropna(subset=['name'])
earnings_2019_df_cleaned = earnings_2019_df_cleaned[~earnings_2019_df_cleaned['name'].isin(['London', 'England', 'Great Britain', 'North East', 'North West', 'Yorkshire and the Humber', 'East Midlands', 'West Midlands', 'East of England', 'South East', 'South West', 'Scotland', 'Wales'])]

# Merge traffic data with geospatial data
merged_earnings_2019_gdf = london_boroughs_gdf.merge(earnings_2019_df_cleaned, left_on='LAD11NM', right_on='name', how='left')

# Convert '2019 Pay (£)' column to numeric, coercing errors to NaN
merged_earnings_2019_gdf['2019 Pay (£)'] = pd.to_numeric(merged_earnings_2019_gdf['2019 Pay (£)'], errors='coerce')

# Drop rows where '2019 Pay (£)' is NaN after conversion
merged_earnings_2019_gdf = merged_earnings_2019_gdf.dropna(subset=['2019 Pay (£)'])

px.choropleth( # plot a choropleth map using the plotly express (px) library
                merged_earnings_2019_gdf, # load the dataframe
                geojson=merged_earnings_2019_gdf.geometry, # Use the geometry column from the merged GeoDataFrame
                locations=merged_earnings_2019_gdf.index, # Use the index as locations since geojson is provided
                color='2019 Pay (£)', # Use the integer column name for 2019
                color_continuous_scale=px.colors.sequential.Viridis, # set the color scale to Viridis, a commonly used color scale
                range_color=[merged_earnings_2019_gdf['2019 Pay (£)'].min(), merged_earnings_2019_gdf['2019 Pay (£)'].max()], # Set range dynamically
                hover_name="name", # Show borough name on hover
                title="Income in London Boroughs (2019)",
                height=700)